In [2]:
import numpy as np

def gen_random_conf(n):
    #input: n = length of array = width of array
    #output: random 10x10 array (-1/2's and +1/2's)
    
    A = np.random.choice([-0.5,0.5], size=(n,n))
    
    return(A)

In [ ]:
#Compute energy as a function of J (i.e. assume J = 1)
# This is inefficient but I think easier to understand 
# Loop over all spins
def calculate_energy(N, spin):
        
    energy = 0
    
    for i in range(N):  # 0, N-1
      for j in range(N): # 0, N-1
        # calculate four interactions
        for delta_i in range(-1,2,2):  # -1 and 1
          ii = i + delta_i
          if (ii < 0): 
            ii = ii + N
          if (ii >=N): 
            ii = ii - N
          energy = energy - spin[i,j]*spin[ii, j] #spin--> array name
        
        for delta_j in range(-1,2,2):  # -1 and 1
          jj = j + delta_j
          if (jj < 0): 
            jj = jj + N
          if (jj >=N): 
            jj = jj - N
          energy = energy - spin[i,j]*spin[i, jj] 
        
        energy = energy * 0.5
    

    return energy

In [3]:
def magnetism_sum(spin):
    magnetism = np.sum(spin)
    
    return magnetism

In [ ]:
def metropolis(N, spin):
    k = 8.617E-5 #boltzman constant (eV/K)
    T = 300 #K
    for i in range(N):
        for j in range(N):
            #random index
            a = np.random.randint(0, N) 
            b = np.random.randint(0, N)
            random =  spin[a, b]
            
            #recalculate energy a
            for delta_a in range(-1,2,2):  # -1 and 1
                aa = a + delta_a
                if (a < 0): 
                    aa = aa + N
                if (aa >=N): 
                    aa = aa - N
                new_energy = new_energy - spin[a,b]*spin[aa, b] #spin--> array name
        
            for delta_b in range(-1,2,2):  # -1 and 1
                bb = b + delta_b
                if (bb < 0): 
                    bb = bb + N
                if (bb >=N): 
                    bb = bb - N
            new_energy = new_energy - spin[a,b]*spin[a, bb] 
        
            new_energy = new_energy * 0.5
            
            if new_energy < 0:
                random *= -1
            #if new system has higher energy, spin accepted with given probability 
            elif rand() < np.exp(-new_energy*k*T):
                random *= -1
            spin[a, b] = random
    return spin